<a href="https://colab.research.google.com/github/kuldeep0009-hub/Deep-learning/blob/main/hyperparamter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [91]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
from scikeras.wrappers import KerasClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

In [92]:
df=pd.read_csv('Churn_Modelling.csv')

In [93]:
df=df.drop(['RowNumber','CustomerId','Surname'],axis=1)

label_encode=LabelEncoder()
df['Gender']=label_encode.fit_transform(df['Gender'])
one_hot=OneHotEncoder(handle_unknown='ignore')
geo_encode=one_hot.fit_transform(df[['Geography']]).toarray()
geo_encode_df=pd.DataFrame(geo_encode,columns=one_hot.get_feature_names_out(['Geography']))

df=pd.concat([df.drop('Geography',axis=1),geo_encode_df],axis=1)

y=df['Exited']
x=df.drop(['Exited'],axis=1)
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20,random_state=42)

scaler=StandardScaler()
x_train=scaler.fit_transform(x_train)
x_test=scaler.transform(x_test)

In [94]:

import pickle
with open('label_encoder.pkl','wb') as file:
  pickle.dump(label_encode,file)
with open('one_hott.pkl','wb') as file:
  pickle.dump(one_hot,file)
with open('scalerr.pkl','wb') as file:
  pickle.dump(scaler,file)

In [95]:
#define a function to create the model and try different parameters(KerasClassifier)

def create_model(neurons=32,layers=1):
  model=Sequential()
  model=model.add(Dense(neurons,activation='relu',input_shape=(x_train.shape[1],)))

  for _ in range(layers-1):
    model.add(Dense(neurons,activation='relu'))
  model.add(Dense(1,activation='sigmoid'))
  model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
  return model



In [102]:
##create keras classifier
model=KerasClassifier(build_fn=create_model,verbose=0)

In [103]:
## define grid search parameters

param_grid={
    'neurons':[16, 32, 64, 128],
    'layers':[1, 2],
    'epochs':[50, 100]
}

In [104]:
grid=GridSearchCV(estimator=model,param_grid=param_grid,n_jobs=-1,cv=3,verbose=1)
grid_result=grid.fit(x_train,y_train)

print('Best :%f using %s'%(grid_result.best_score_,grid_result.best_params_))

AttributeError: 'super' object has no attribute '__sklearn_tags__'